##Final Submission With Two Function

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pylab 
import scipy.stats as stats
import datetime
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm 
import gc
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import log_loss
import lightgbm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from prettytable import PrettyTable
import prettytable
import pickle
from sklearn.model_selection import GridSearchCV, cross_val_score, cross_val_predict, train_test_split
from tqdm import tqdm

In [ ]:
def reduce_mem_usage(df, verbose=True):
  '''This function helps reduce memmory occuipied by 
  convertig the value to its rqquired data type'''

  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2    
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)  
          else:
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)    
  end_mem = df.memory_usage().sum() / 1024**2
  if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  return df

In [ ]:
from tqdm import tqdm
def oneHotEncoding_mod(df, features,transaction):
  hist_transac_dict = {'category_2':[1.0,2.0,3.0,4.0,5.0], 'category_3':[1.0,2.0,3.0], 'month_lag':[0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10,-11,-12,-13]} 
  new_transac_dict = {'category_2':[1.0,2.0,3.0,4.0,5.0], 'category_3':[1.0,2.0,3.0], 'month_lag':[1,2]} 

  for feat in tqdm(features):
    if (transaction=='hist'):
      dict_=hist_transac_dict
    else:
      dict_=new_transac_dict
      
    unique_values =dict_[feat]

    for cat in unique_values:
      df[feat+'={}'.format(cat)] = (df[feat] == cat).astype(int)

In [ ]:
def fillup_null_inf_values(train,df_train_columns):
  for col in df_train_columns:
    train[col].fillna(train[col].median(),inplace=True)

  train = train.replace(np.inf, np.nan)
  col=['new_hist_purchase_amount_sum','new_hist_purchase_amount_min','new_hist_purchase_amount_mean',
     'new_hist_purchase_amount_max','new_hist_purchase_amount_var','new_hist_purchase_amount_diff','purchase_amount_total']

  for c in col:
    train[c].fillna(train[c].median(),inplace=True)
  return train

In [ ]:
def get_new_columns(name,aggs):
    return [name + '_' + k + '_' + agg for k in aggs.keys() for agg in aggs[k]]

In [ ]:
## reference: https://towardsdatascience.com/find-your-best-customers-with-customer-segmentation-in-python-61d602f9eee6

def RScore(x,p,d):
    if x <= d[p][0.011]:
        return 1
    elif x <= d[p][0.050]:
        return 2
    elif x <= d[p][0.25]: 
        return 3
    elif x <= d[p][0.5]:
        return 4
    elif x <= d[p][0.75]:
        return 5
    elif x <= d[p][0.95]:
        return 6
    elif x <= d[p][0.989]:
        return 7
    else:
        return 8
    
def FMScore(x,p,d):
    if x <= d[p][0.011]:
        return 8
    elif x <= d[p][0.050]:
        return 7
    elif x <= d[p][0.25]: 
        return 6
    elif x <= d[p][0.5]:
        return 5
    elif x <= d[p][0.75]:
        return 4
    elif x <= d[p][0.95]:
        return 3
    elif x <= d[p][0.989]:
        return 2
    else:
        return 1

In [ ]:
def rfm(df,quantiles, transc):
  ## grouping quantiles
  df[transc+'r_quantile'] = df[transc+'purchase_recency'].apply(RScore, args=(transc+'purchase_recency',quantiles))
  df[transc+'f_quantile'] = df[transc+'card_id_size'].apply(FMScore, args=(transc+'card_id_size',quantiles))
  df[transc+'m_quantile'] = df[transc+'purchase_amount_sum'].apply(FMScore, args=(transc+'purchase_amount_sum',quantiles))
  ## calaculating RFM index and RFMScore
  df[transc+'RFMindex'] = df[transc+'r_quantile'].map(str)+df[transc+'f_quantile'].map(str)+df[transc+'m_quantile'].map(str)                       
  df[transc+'RFMScore'] = df[transc+'r_quantile']+df[transc+'f_quantile']+df[transc+'m_quantile']

In [ ]:
#definind the rmse metric
import keras.backend as K
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

In [3]:
test=pd.read_csv("/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_1/DATA/test_engineered_4_Jul_2021.csv",nrows=10000)

In [4]:
df_test_columns = [c for c in test.columns if c not in ['card_id', 'first_active_month', 'target', 'outliers']]  
card_dict=dict()
for c_id in tqdm(test['card_id']):
  tmp_df=test[test['card_id']==c_id][df_test_columns]
  tmp_df=tmp_df.values.tolist()[0]
  card_dict[c_id]=tmp_df
  #card_dict


100%|██████████| 10000/10000 [00:26<00:00, 383.44it/s]


In [5]:
a_file = open("/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_1/DATA/card_dict_top_10K.pkl", "wb")
pickle.dump(card_dict, a_file)
a_file.close()

In [ ]:
card_dict_file = open("/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_1/DATA/card_dict_top_10K.pkl", "rb")
card_dict = pickle.load(card_dict_file)

##final_fun_1():


In [6]:
%%writefile EMCR.py
import pickle
import numpy as np
import streamlit as st
import lightgbm

st.title("Elo Merchant Category Recommendation - Kaggle Competition with Loyalty Score Predictions")

card_dict_file = open("/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_1/DATA/card_dict.pkl", "rb")
card_dict = pickle.load(card_dict_file)
Pkl_Filename = "/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_1/DATA/LGB_Model_61579.pkl"  

def final_fun_1(card_id='C_ID_0ab67a22ab'):
  card_data=card_dict.get(card_id)
  card_data=np.array(card_data).reshape((1,-1))
  pred_y_test_pickle=lightbgm_reg_pickle.predict(card_data)
  return pred_y_test_pickle[0]

# Load the Model back from file 
with open(Pkl_Filename, 'rb') as file:
  lightbgm_reg_pickle = pickle.load(file)
  card_select = st.selectbox('Select Card ID: ',card_dict.keys())
  card_selection_msg=st.text('You have selected card: '+card_select)
  if st.button('Predict Loyalty Score'):
    loyalty_score=final_fun_1(card_select)
    st.write('Card Id: '+card_select)
    st.write("Loylty Score: "+str(loyalty_score))


Writing EMCR.py


##final_fun_2():

In [ ]:
def final_fun_2(train,historical_transactions,new_merchant_transctions):
  #one hot encoding
  categorical_features = ['category_2','category_3','month_lag']
  oneHotEncoding_mod(historical_transactions, features=categorical_features,transaction='hist')
  oneHotEncoding_mod(new_merchant_transctions, features=categorical_features,transaction='new')
  
  new_merchant_transctions['purchase_amount'] = np.round(new_merchant_transctions['purchase_amount'] / 0.00150265118 + 497.06, 2)
  historical_transactions['purchase_amount'] = np.round(historical_transactions['purchase_amount'] / 0.00150265118 + 497.06, 2)
  historical_transactions=reduce_mem_usage(historical_transactions)
  new_merchant_transctions=reduce_mem_usage(new_merchant_transctions)
  gc.collect()
  for df in [historical_transactions,new_merchant_transctions]:
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['year'] = df['purchase_date'].dt.year
    df['day']=df['purchase_date'].dt.day
    df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['month'] = df['purchase_date'].dt.month
    df['dayofweek'] = df['purchase_date'].dt.dayofweek
    df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
    df['hour'] = df['purchase_date'].dt.hour
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['installments']=df['installments'].map({-1:14, 0:0,1:1,2:2,3:3,4:4,5:5,6:6,7:7,8:8,9:9,10:10,11:11,12:12,999:13})
    df['price']=df['purchase_amount']/df['installments']
    df['purchase_amount_quantiles']= pd.qcut(df['purchase_amount'], 5, labels=False)
    df['installments_quantiles']= pd.qcut(df['installments'], 5, labels=False,duplicates='drop')
    df['amount_month_ratio'] = df['purchase_amount'].values / (1.0 + df['month_diff'].values)
    df=reduce_mem_usage(df)
    gc.collect()
  #Holidays features
  holidays = {'EasterDay_2017' : '2017-04-16','AllSoulsDay_2017': '2017-11-2','ChristmasDay_2017': '2017-12-25','FathersDay_2017': '2017-08-13',
            'ChildrenDay_2017':'2017-10-12','BlackFriday_2017':'2017-11-24','ValentineDay_2017':'2017-06-12','MothersDay_2018':'2018-05-13'}
  for day, date in holidays.items():
    new_merchant_transctions[day] = (pd.to_datetime(date) - new_merchant_transctions['purchase_date']).dt.days
    new_merchant_transctions[day] = new_merchant_transctions[day].apply(lambda x: x if x > 0 and x < 30 else 0)
    historical_transactions[day] = (pd.to_datetime(date) - historical_transactions['purchase_date']).dt.days
    historical_transactions[day] = historical_transactions[day].apply(lambda x: x if x > 0 and x < 30 else 0)
  
  
  aggs = {}
  for col in ['month','hour','day','weekend','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id','price','city_id','state_id',
            'category_2=1.0', 'category_2=3.0', 'category_2=5.0',
            'category_2=2.0', 'category_2=4.0', 'category_3=1.0', 'category_3=2.0',
            'category_3=3.0', 'month_lag=-8', 'month_lag=-7', 'month_lag=-6',
            'month_lag=-5', 'month_lag=-11', 'month_lag=0', 'month_lag=-3',
            'month_lag=-9', 'month_lag=-4', 'month_lag=-1', 'month_lag=-13',
            'month_lag=-10', 'month_lag=-12', 'month_lag=-2']:
      aggs[col] = ['nunique']

  aggs['EasterDay_2017']=['sum', 'mean']
  aggs['AllSoulsDay_2017']=['sum', 'mean']
  aggs['ChristmasDay_2017']=['sum', 'mean']
  aggs['FathersDay_2017']=['sum', 'mean']
  aggs['ChildrenDay_2017']=['sum', 'mean']
  aggs['BlackFriday_2017']=['sum', 'mean']
  aggs['ValentineDay_2017']=['sum', 'mean']
  aggs['MothersDay_2018']=['sum', 'mean']

  aggs['category_2=1.0']=['sum', 'mean']
  aggs['category_2=3.0']=['sum', 'mean']
  aggs['category_2=2.0']=['sum', 'mean']
  aggs['category_2=4.0']=['sum', 'mean']
  aggs['category_2=5.0']=['sum', 'mean']
  aggs['category_3=1.0']=['sum', 'mean']
  aggs['category_3=2.0']=['sum', 'mean']
  aggs['category_3=3.0']=['sum', 'mean']

  aggs['month_lag=0']= ['sum','mean']
  aggs['month_lag=-1']=['sum','mean']
  aggs['month_lag=-2']=['sum','mean']

  aggs['purchase_amount'] = ['sum','max','min','mean','var']
  aggs['installments'] = ['sum','max','min','mean','var','std','skew']
  aggs['purchase_date'] = ['max','min']
  aggs['month_lag'] = ['mean', 'std', 'min', 'max', 'skew']
  aggs['month_diff'] = ['mean']
  aggs['authorized_flag'] = ['sum', 'mean']
  aggs['weekend'] = ['sum', 'mean']
  aggs['category_1'] = ['sum', 'mean']
  aggs['card_id'] = ['size']
  aggs['installments_quantiles']=['var', 'mean', 'skew']
  aggs['purchase_amount_quantiles']=['var', 'mean', 'skew']
  aggs['amount_month_ratio']=['mean', 'std', 'min', 'max', 'skew']
  #aggs['price'] = ['sum', 'mean']  
  aggs['weekofyear']=['mean', 'min', 'max']
  aggs['dayofweek']=['mean', 'min', 'max']
  aggs['hour']=['mean', 'min', 'max']  
    
  new_columns = get_new_columns('hist',aggs)
  df_hist_trans_group = historical_transactions.groupby('card_id').agg(aggs)
  df_hist_trans_group.columns = new_columns
  df_hist_trans_group.reset_index(drop=False,inplace=True)

  df_hist_trans_group['hist_purchase_date_diff'] = (df_hist_trans_group['hist_purchase_date_max'] - df_hist_trans_group['hist_purchase_date_min']).dt.days
  df_hist_trans_group['hist_purchase_date_average'] = df_hist_trans_group['hist_purchase_date_diff']/df_hist_trans_group['hist_card_id_size']
  df_hist_trans_group['hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['hist_purchase_date_max']).dt.days
  df_hist_trans_group['hist_of_hist_purchase_date_average']=df_hist_trans_group['hist_purchase_date_diff']*df_hist_trans_group['hist_purchase_date_average']

  #Newly added features
  df_hist_trans_group['hist_purchase_amount_diff'] = df_hist_trans_group['hist_purchase_amount_max'].values - df_hist_trans_group['hist_purchase_amount_min'].values
  df_hist_trans_group['hist_purchase_count_ratio'] = df_hist_trans_group['hist_card_id_size'].values / (1.0 + df_hist_trans_group['hist_purchase_date_diff'].values)
  df_hist_trans_group['hist_purchase_recency'] = (datetime.datetime.today() - pd.to_datetime(df_hist_trans_group['hist_purchase_date_max']))/(24*np.timedelta64(1, 'h'))

  df_hist_trans_group['hist_category_2_pa_mean'] = historical_transactions.groupby('category_2')['purchase_amount'].transform('mean')
  df_hist_trans_group['hist_category_3_pa_mean'] = historical_transactions.groupby('category_3')['purchase_amount'].transform('mean')

  df_hist_trans_group['hist_merchant_id_count_mean'] = df_hist_trans_group['hist_card_id_size'].values / (1.0+df_hist_trans_group['hist_merchant_id_nunique'].values)
  df_hist_trans_group['hist_month_lag_0_-1_ratio'] = df_hist_trans_group['hist_month_lag=0_sum']/ (1.0+ df_hist_trans_group['hist_month_lag=-1_sum'])
  df_hist_trans_group['hist_month_lag_0_-2_ratio'] = df_hist_trans_group['hist_month_lag=0_sum'] / (1.0+ df_hist_trans_group['hist_month_lag=-2_sum'])

  train = train.merge(df_hist_trans_group,on='card_id',how='left')
  del df_hist_trans_group
  gc.collect()


  aggs = {}
  for col in ['month','hour','day','weekend','weekofyear','dayofweek','year','subsector_id','merchant_id','merchant_category_id','price','city_id','state_id',
            'category_2=1.0', 'category_2=3.0', 'category_2=2.0',
            'category_2=4.0', 'category_2=5.0', 'category_3=2.0', 'category_3=1.0',
            'category_3=3.0', 'month_lag=1', 'month_lag=2']:
      aggs[col] = ['nunique']

  aggs['EasterDay_2017']=['sum', 'mean']
  aggs['AllSoulsDay_2017']=['sum', 'mean']
  aggs['ChristmasDay_2017']=['sum', 'mean']
  aggs['FathersDay_2017']=['sum', 'mean']
  aggs['ChildrenDay_2017']=['sum', 'mean']
  aggs['BlackFriday_2017']=['sum', 'mean']
  aggs['ValentineDay_2017']=['sum', 'mean']
  aggs['MothersDay_2018']=['sum', 'mean']

  aggs['category_2=1.0']=['sum', 'mean']
  aggs['category_2=3.0']=['sum', 'mean']
  aggs['category_2=2.0']=['sum', 'mean']
  aggs['category_2=4.0']=['sum', 'mean']
  aggs['category_2=5.0']=['sum', 'mean']
  aggs['category_3=1.0']=['sum', 'mean']
  aggs['category_3=2.0']=['sum', 'mean']
  aggs['category_3=3.0']=['sum', 'mean']

  aggs['month_lag=1']= ['sum','mean']
  aggs['month_lag=2']=['sum','mean']

  aggs['purchase_amount'] = ['sum','max','min','mean','var']
  aggs['installments'] = ['sum','max','min','mean','var','std','skew']
  aggs['purchase_date'] = ['max','min']
  aggs['month_lag'] = ['mean', 'std', 'min', 'max', 'skew']
  aggs['month_diff'] = ['mean']
  aggs['authorized_flag'] = ['sum', 'mean']
  aggs['weekend'] = ['sum', 'mean']
  aggs['category_1'] = ['sum', 'mean']
  aggs['card_id'] = ['size']
  aggs['installments_quantiles']=['var', 'mean', 'skew']
  aggs['purchase_amount_quantiles']=['var', 'mean', 'skew']
  #aggs['price'] = ['sum', 'mean']
  aggs['weekofyear']=['mean', 'min', 'max']
  aggs['dayofweek']=['mean', 'min', 'max']
  aggs['hour']=['mean', 'min', 'max']

  new_columns = get_new_columns('new_hist',aggs)
  df_hist_trans_group = new_merchant_transctions.groupby('card_id').agg(aggs)
  df_hist_trans_group.columns = new_columns
  df_hist_trans_group.reset_index(drop=False,inplace=True)
  df_hist_trans_group['new_hist_purchase_date_diff'] = (df_hist_trans_group['new_hist_purchase_date_max'] - df_hist_trans_group['new_hist_purchase_date_min']).dt.days
  df_hist_trans_group['new_hist_purchase_date_average'] = df_hist_trans_group['new_hist_purchase_date_diff']/df_hist_trans_group['new_hist_card_id_size']
  df_hist_trans_group['new_hist_purchase_date_uptonow'] = (datetime.datetime.today() - df_hist_trans_group['new_hist_purchase_date_max']).dt.days
  df_hist_trans_group['new_hist_of_new_hist_purchase_date_average']=df_hist_trans_group['new_hist_purchase_date_diff']*df_hist_trans_group['new_hist_purchase_date_average']

  #newly added fature
  df_hist_trans_group['new_hist_purchase_amount_diff'] = df_hist_trans_group['new_hist_purchase_amount_max'].values - df_hist_trans_group['new_hist_purchase_amount_min'].values
  df_hist_trans_group['new_hist_purchase_count_ratio'] = df_hist_trans_group['new_hist_card_id_size'].values / (1.0 + df_hist_trans_group['new_hist_purchase_date_diff'].values)
  df_hist_trans_group['new_hist_purchase_recency'] = (datetime.datetime.today() - pd.to_datetime(df_hist_trans_group['new_hist_purchase_date_max']))/(24*np.timedelta64(1, 'h'))

  df_hist_trans_group['new_hist_category_2_pa_mean'] = historical_transactions.groupby('category_2')['purchase_amount'].transform('mean')
  df_hist_trans_group['new_hist_category_3_pa_mean'] = historical_transactions.groupby('category_3')['purchase_amount'].transform('mean')

  df_hist_trans_group['new_hist_merchant_id_count_mean'] = df_hist_trans_group['new_hist_card_id_size'].values / (1.0+df_hist_trans_group['new_hist_merchant_id_nunique'].values)
  df_hist_trans_group['new_hist_month_lag_1_2_ratio'] = df_hist_trans_group['new_hist_month_lag=1_sum'] / (1.0+ df_hist_trans_group['new_hist_month_lag=2_sum'])

  train = train.merge(df_hist_trans_group,on='card_id',how='left')
  del df_hist_trans_group;
  gc.collect()

  
  ## qunatiles of RFM
  quantiles_new = train[['new_hist_purchase_recency','new_hist_card_id_size','new_hist_purchase_amount_sum']].quantile(q=[0.011,0.05,0.25,0.5,0.75,0.95,0.989]).to_dict()
  quantiles_hist = train[['hist_purchase_recency','hist_card_id_size','hist_purchase_amount_sum']].quantile(q=[0.011,0.05,0.25,0.5,0.75,0.95,0.989]).to_dict()
  
  train['outliers'] = 0
  train.loc[train['target'] < -30, 'outliers'] = 1

  for df in [train]:
    rfm(df,quantiles_new,transc = 'new_hist_')
    rfm(df,quantiles_hist,transc = 'hist_')
    df['new_hist_RFMindex']=df["new_hist_RFMindex"].map(int)
    df["hist_RFMindex"]=df["hist_RFMindex"].map(int)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['dayofweek'] = df['first_active_month'].dt.dayofweek
    df['weekofyear'] = df['first_active_month'].dt.weekofyear
    df['month'] = df['first_active_month'].dt.month
    df['elapsed_time'] = (datetime.datetime.today() - df['first_active_month']).dt.days
    df['hist_first_buy'] = (df['hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['new_hist_first_buy'] = (df['new_hist_purchase_date_min'] - df['first_active_month']).dt.days
    df['card_id_total'] = df['new_hist_card_id_size']+df['hist_card_id_size']
    df['purchase_amount_total'] = df['new_hist_purchase_amount_sum']+df['hist_purchase_amount_sum']
    df['days_feature1'] = df['elapsed_time'] * df['feature_1']

    for f in ['hist_purchase_date_max','hist_purchase_date_min','new_hist_purchase_date_max','new_hist_purchase_date_min']:
      df[f] = df[f].fillna(df[f].mean())
      df[f] = df[f].astype(np.int64) * 1e-9
    
    for f in ['feature_1','feature_2','feature_3']:
      order_label = train.groupby([f])['outliers'].mean()
      df[f] = df[f].map(order_label)


  df_train_columns = [c for c in train.columns if c not in ['card_id', 'first_active_month', 'target', 'outliers']]
  train=fillup_null_inf_values(train,df_train_columns)
  
  y = train['target']
  X = train[df_train_columns]
  X_train_all, X_test_com, y_train_all, y_test_com = train_test_split(X, y, test_size=0.20, random_state=42)
 
  Pkl_Filename = "/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_1/DATA/LGB_Model_61579.pkl"  
  # Load the Model back from file
  with open(Pkl_Filename, 'rb') as file:
    lightbgm_reg_pickle = pickle.load(file)

  pred_y_test_pickle=lightbgm_reg_pickle.predict(X_test_com)
  print("RMSE Score::{:.3f}".format(rmse(y_test_com, pred_y_test_pickle)))

In [ ]:
#Loading all data files from goole drive
train = reduce_mem_usage(pd.read_csv('/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_1/DATA/train.csv', parse_dates=["first_active_month"]))
historical_transactions=reduce_mem_usage(pd.read_csv("/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_1/DATA/historical_transactions_model_imputed.csv",parse_dates=['purchase_date']))
new_merchant_transctions=reduce_mem_usage(pd.read_csv("/content/drive/MyDrive/CASE_STUDIES/CASE_STUDY_1/DATA/new_merchant_transctions_model_imputed.csv",parse_dates=['purchase_date']))

Mem. usage decreased to  4.04 Mb (56.2% reduction)
Mem. usage decreased to 1193.84 Mb (61.6% reduction)
Mem. usage decreased to 76.76 Mb (63.4% reduction)


In [ ]:
final_fun_2(train,historical_transactions,new_merchant_transctions)

100%|██████████| 3/3 [00:00<00:00, 11.16it/s]


Mem. usage decreased to 1804.64 Mb (70.9% reduction)
Mem. usage decreased to 106.71 Mb (52.9% reduction)
Mem. usage decreased to 2415.44 Mb (46.6% reduction)
Mem. usage decreased to 155.38 Mb (47.8% reduction)
RMSE Score::3.590
